# Auswertung der Stickoxide

Die Stickoxide wurden von unserem Scraper erfasst. Der Endpunkt lautet `https://data.rbb-online.de/nox/data?from=2020-03-03T23%3A00%3A00Z&to=2020-03-19T22%3A59%3A59Z`.

Das Notebook lässt sich über folgenden Befehl ausführen:
```
docker run --rm -p 8889:8888 --ip=0.0.0.0 -v (pwd):/home/jovyan -e CHOWN_HOME=yes -e CHOWN_EXTRA_ARGS=-R --user=root jupyter/scipy-notebook
```

Alle Dependencies, die nicht im `jupyter/scipy-notebook`-Image definiert sind, werden in der nächsten Zeile installiert.

In [1]:
! conda install altair -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.



In [2]:
import pandas as pd
import numpy as np
import altair as alt
import json
import pytz

## Einlesen als Dataframes

In [3]:
def read_nox(path):
    '''
    Reads NOx JSON data that we saved from our endpoint and returns a pandas DataFrame
    '''
    with open(path, 'r') as f:
        response = json.load(f)
    df = pd.DataFrame([(d['id'], d['name'], d['federalstate'], v['t'], v['y']) for d in response['data'] for v in d['values']], columns=['id', 'name', 'federal_state', 'datetime', 'val'])
    df['datetime'] = pd.to_datetime(df['datetime'], utc=True)
    return df

In [4]:
measurements = read_nox('./input/2020-03-03--2020-03-31.json')

# drop last day to make sure we only have complete days
measurements = measurements[measurements['datetime'].dt.dayofyear != measurements['datetime'].max().dayofyear]
measurements.head()

,id,name,federal_state,datetime,val
0,BBI-001,Brandenburg - Flughafen Schönefeld,Brandenburg,2020-03-03 23:00:00+00:00,21
1,BBI-001,Brandenburg - Flughafen Schönefeld,Brandenburg,2020-03-04 00:00:00+00:00,12
2,BBI-001,Brandenburg - Flughafen Schönefeld,Brandenburg,2020-03-04 01:00:00+00:00,12
3,BBI-001,Brandenburg - Flughafen Schönefeld,Brandenburg,2020-03-04 02:00:00+00:00,14
4,BBI-001,Brandenburg - Flughafen Schönefeld,Brandenburg,2020-03-04 03:00:00+00:00,15


In [5]:
# how much data do we have?
measurements.count()

id               27241
name             27241
federal_state    27241
datetime         27241
val              27241
dtype: int64

In [6]:
berlin = measurements[measurements['federal_state'] == 'Berlin']
brandenburg = measurements[measurements['federal_state'] == 'Brandenburg']

In [7]:
berlin.groupby(['name', berlin['datetime'].dt.day]).mean()

val
name                       datetime           
Berlin - Amrumer Straße    3         37.000000
                           4         29.708333
                           5         29.541667
                           6         31.583333
                           7         34.458333
...                                        ...
Berlin - Silbersteinstraße 22        22.333333
                           23        47.625000
                           24        53.772727
                           25        49.619048
                           26        42.300000

[470 rows x 1 columns]

## Plotten

Um einen Eindruck zu erhalten, wie unsere Daten für Berlin und Brandenburg aussehen, plotten wir das tägliche Mittel pro Station:

In [8]:
berlin_daily_avg = berlin.set_index('datetime').groupby('name').resample('D').mean().reset_index()
berlin_daily_avg.head()

,name,datetime,val
0,Berlin - Amrumer Straße,2020-03-03 00:00:00+00:00,37.000000
1,Berlin - Amrumer Straße,2020-03-04 00:00:00+00:00,29.708333
2,Berlin - Amrumer Straße,2020-03-05 00:00:00+00:00,29.541667
3,Berlin - Amrumer Straße,2020-03-06 00:00:00+00:00,31.583333
4,Berlin - Amrumer Straße,2020-03-07 00:00:00+00:00,34.458333


In [9]:
alt.Chart(berlin_daily_avg).mark_line(interpolate='basis').encode(
    x='datetime:T',
    y='val:Q',
    color='name:N'
)

alt.Chart(...)

In [10]:
brandenburg_daily_avg = brandenburg.set_index('datetime').groupby('name').resample('D').mean().reset_index()
brandenburg_daily_avg.head()

,name,datetime,val
0,Brandenburg - Bernau - Lohmühlenstraße,2020-03-03 00:00:00+00:00,17.000000
1,Brandenburg - Bernau - Lohmühlenstraße,2020-03-04 00:00:00+00:00,31.416667
2,Brandenburg - Bernau - Lohmühlenstraße,2020-03-05 00:00:00+00:00,28.708333
3,Brandenburg - Bernau - Lohmühlenstraße,2020-03-06 00:00:00+00:00,30.875000
4,Brandenburg - Bernau - Lohmühlenstraße,2020-03-07 00:00:00+00:00,13.791667


In [11]:
alt.Chart(brandenburg_daily_avg).mark_line(interpolate='basis').encode(
    x='datetime:T',
    y='val:Q',
    color='name:N'
)

alt.Chart(...)

Es scheint nicht so, als ob sich für die letzten zwei Wochen ein klarer Trend abzeichnet. Was man jetzt noch prüfen kann:

- Wie war das Wetter in den letzten zwei Wochen?
- Wie sieht die Stickstoffbelastung während der Spitzenzeiten aus?

## Spitzenzeiten

Als Spitzenzeiten gelten Zeiten zwischen 6 und 9 Uhr sowie 16 und 19 Uhr ([siehe Wikipedia](https://de.wikipedia.org/wiki/Verkehrszeiten#Hauptverkehrszeit)).

In [12]:
def in_rush_hour(df):
    # we have to reconvert it to get the correct mask across summer and winter time
    converted = df['datetime'].dt.tz_convert('Europe/Berlin')
    h = converted.dt.hour
    return df[((h >= 6) & (h < 9)) | ((h >= 16) & (h < 19))]

In [13]:
berlin_rush_hour = in_rush_hour(berlin)
berlin_rush_hour = berlin_rush_hour.set_index('datetime').groupby('name').resample('D').mean().reset_index()

In [14]:
alt.Chart(berlin_rush_hour).mark_line(interpolate='basis').encode(
    x='datetime:T',
    y='val:Q',
    color='name:N'
)

alt.Chart(...)

In [15]:
brandenburg_rush_hour = in_rush_hour(brandenburg)
brandenburg_rush_hour = brandenburg_rush_hour.set_index('datetime').groupby('name').resample('D').mean().reset_index()

In [16]:
alt.Chart(brandenburg_rush_hour).mark_line(interpolate='basis').encode(
    x='datetime:T',
    y='val:Q',
    color='name:N'
)

alt.Chart(...)

Die Werte sind zwar etwas höher (wie erwartet), aber auch hier ergibt sich keine deutliche Änderung in jüngerer Vergangenheit. Wir versuchen es noch mal mit dem drei- und siebentägigen Mittel:

In [17]:
def n_day_avg(df, n_days):
    '''
    Calculates a rolling average over n days
    '''
    return df.set_index('datetime').resample('D').mean().rolling(n_days).mean().dropna().reset_index()

In [18]:
three_day_avg = n_day_avg(berlin, 3)
seven_day_avg = n_day_avg(berlin, 7)

alt.hconcat(
    alt.Chart(three_day_avg).mark_line(interpolate='basis').encode(x='datetime', y='val'),
    alt.Chart(seven_day_avg).mark_line(interpolate='basis').encode(x='datetime', y='val')
)

alt.HConcatChart(...)

Auch nicht.

Es gibt [Studien](https://www.sciencedirect.com/science/article/pii/S0048969717319988), die [versuchen](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3856232/) Verkehr mithilfe von Luftverschmutzung zu schetzen und auch wenn sie vielversprechend sind (also hohe Korellationen von Vorhersagen und tatsächlichen Werten zeigen), gibt es keine, die das zuverlässig auf einem engen Raum tun.

Die [Esa Animation](https://www.esa.int/ESA_Multimedia/Videos/2020/03/Coronavirus_nitrogen_dioxide_emissions_drop_over_Italy) hat einen räumlich sehr grobe Auflösung - ganz Europa - zur Grundlage genommen und darauf einen vierzehntägen Durchschnitt berechnet um den Rückgang von Stickoxid in Zusammenhang mit den Ausgangssperren zu zeigen.

## Außerhalb der Rush Hour

In [19]:
be_not_in_rush_hour = berlin[~berlin.index.isin(berlin_rush_hour.index)]
be_not_in_rush_hour = be_not_in_rush_hour.set_index('datetime').groupby('name').resample('D').mean().reset_index()
be_not_in_rush_hour.head()

,name,datetime,val
0,Berlin - Amrumer Straße,2020-03-03 00:00:00+00:00,37.000000
1,Berlin - Amrumer Straße,2020-03-04 00:00:00+00:00,29.708333
2,Berlin - Amrumer Straße,2020-03-05 00:00:00+00:00,29.541667
3,Berlin - Amrumer Straße,2020-03-06 00:00:00+00:00,31.583333
4,Berlin - Amrumer Straße,2020-03-07 00:00:00+00:00,34.458333


In [20]:
alt.Chart(be_not_in_rush_hour).mark_line(interpolate='basis').encode(
    x='datetime:T',
    y='val:Q',
    color='name:N'
)

alt.Chart(...)

In [21]:
bb_not_in_rush_hour = brandenburg[~brandenburg.index.isin(brandenburg_rush_hour.index)]
bb_not_in_rush_hour = bb_not_in_rush_hour.set_index('datetime').groupby('name').resample('D').mean().reset_index()
bb_not_in_rush_hour.head()

,name,datetime,val
0,Brandenburg - Bernau - Lohmühlenstraße,2020-03-03 00:00:00+00:00,17.000000
1,Brandenburg - Bernau - Lohmühlenstraße,2020-03-04 00:00:00+00:00,31.416667
2,Brandenburg - Bernau - Lohmühlenstraße,2020-03-05 00:00:00+00:00,28.708333
3,Brandenburg - Bernau - Lohmühlenstraße,2020-03-06 00:00:00+00:00,30.875000
4,Brandenburg - Bernau - Lohmühlenstraße,2020-03-07 00:00:00+00:00,13.791667


In [22]:
alt.Chart(bb_not_in_rush_hour).mark_line(interpolate='basis').encode(
    x='datetime:T',
    y='val:Q',
    color='name:N'
)

alt.Chart(...)

## Lineare Regression

In [23]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression

### Tägliches Mittel

In [24]:
daily_avg = berlin.set_index('datetime').resample('D').mean().reset_index()
daily_avg.head()

,datetime,val
0,2020-03-03 00:00:00+00:00,27.812500
1,2020-03-04 00:00:00+00:00,32.493473
2,2020-03-05 00:00:00+00:00,28.478372
3,2020-03-06 00:00:00+00:00,30.398034
4,2020-03-07 00:00:00+00:00,21.965686


In [25]:
x = daily_avg['datetime'].values.astype('int64').reshape(-1, 1)
y = daily_avg['val'].values.reshape(-1, 1)
linear_regressor = LinearRegression()
linear_regressor.fit(x, y)
linear_regressor.score(x, y)

0.22616829281329953

$R^2$ ist fast 0, also gibt es in unserem Datensatz kaum eine Korrelation zwischen Datum und Stickstoffausstoß,

In [26]:
daily_avg.loc[:,'prediction'] = linear_regressor.predict(x)

In [29]:
alt.Chart(daily_avg).mark_line().encode(x='datetime', y='prediction')

alt.Chart(...)

### Mittel Außerhalb der Stoßzeiten

In [30]:
x = be_not_in_rush_hour['datetime'].values.astype('int64').reshape(-1, 1)
y = be_not_in_rush_hour['val'].values.reshape(-1, 1)
linear_regressor = LinearRegression()
linear_regressor.fit(x, y)
linear_regressor.score(x, y)

0.07633698755471097

In [31]:
be_not_in_rush_hour.loc[:,'prediction'] = linear_regressor.predict(x)

In [32]:
alt.Chart(be_not_in_rush_hour).mark_line().encode(x='datetime', y='prediction')

alt.Chart(...)

### Mittel in den Stoßzeiten

In [34]:
x = berlin_rush_hour['datetime'].values.astype('int64').reshape(-1, 1)
y = berlin_rush_hour['val'].values.reshape(-1, 1)
linear_regressor = LinearRegression()
linear_regressor.fit(x, y)
linear_regressor.score(x, y)

0.046211723987714115

In [36]:
berlin_rush_hour.loc[:,'prediction'] = linear_regressor.predict(x)
alt.Chart(berlin_rush_hour).mark_line().encode(x='datetime', y='prediction')

alt.Chart(...)

## Weitere Gedanken

- Es gibt eine Studie, die Feinstaub (PM2.5) mit einer bestimmten Menge an Zigaretten gleichsetzt. Faustregel: 22 µg PM2.5 $\approx$ 1 Zigarette [Link](http://berkeleyearth.org/air-pollution-and-cigarette-equivalence/)

- Rumgeschwanke von normalen Wochen / vergleichbar mit Geschwanke vor Homeoffice (stddev / Fluktuation)
- Eventuell brauche ich gar keine Trainingsdaten? Man will ja nur 'nen normalen Fit
- Wann haben die Leute auf Homeoffice gesetzt?

- Die drei niedrigsten Stickoxiddaten in Berlin waren letzte Woche (Christopher)

### TODO

- Man sollte die höchsten / niedrigsten Werte isolieren und die Effekte da messen
- Wir sollten schauen, ob wir für den gesamten Zeitraum Messwerte haben